In [116]:
import requests
import json
import pandas as pd
import time

In [117]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, indent=4)
    print(text)

## Offene Gesetze

Let's down load all the metadata for all the laws! 

Saving it into CSV and dataframe for now

Later on need to write a function to grab updates from the api but this corpus is enough for our prototyping needs

In [118]:
# of_response = [requests.get("https://api.offenegesetze.de/v1/veroeffentlichung/?year="+str(year)) for year in range(1949,2022)]

In [ ]:
x = 0
og_response = []
og_response.append(requests.get("https://api.offenegesetze.de/v1/veroeffentlichung/"))
next_page = og_response[-1].json()["next"]
while next_page != "None":
    og_response.append(requests.get(next_page))
    next_page = og_response[-1].json()["next"]
    x += 1
    print(
        next_page, x
    )  # added for testing today because there's an API malefunction. remove when things get back to normal
    if x % 15 == 0:
        time.sleep(1)

In [135]:
# OffeneGesetze_df_list = [pd.DataFrame.from_dict(of_response[year].json()["results"]) for year in range(0,73)]
OffeneGesetze_df_list = [
    pd.DataFrame.from_dict(og_response[page].json()["results"])
    for page in range(0, x + 1)
]

In [136]:
OffeneGesetze_df = pd.concat(OffeneGesetze_df_list, ignore_index=True)

In [140]:
OffeneGesetze_df.to_csv(
    index=False, path_or_buf="../data/source_laws/OffeneGesetzeAPI.csv"
)

In [ ]:
# list of dictionaries, can be converted ot pandas dataframe with from_dict - Done above yay!
jprint(of_response[72].json()["results"])

In [ ]:
# just a quick check to see if I have reasonable amounts of data for each year
[len(OffeneGesetze_df[OffeneGesetze_df["year"] == yr]) for yr in range(1949, 2022)]

In [141]:
OffeneGesetze_df.head()

,id,kind,year,number,date,url,api_url,document_url,order,title,law_date,page,pdf_page,num_pages
0,bgbl1-2021-74-1,bgbl1,2021,74,2021-10-19T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,1,Dreiundsechzigste Verordnung zur Durchführung ...,2021-10-07T00:00:00Z,4682.0,2,1
1,bgbl1-2021-74-2,bgbl1,2021,74,2021-10-19T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,2,Zweite Verordnung zur Änderung der Weinverordn...,2021-10-11T00:00:00Z,4683.0,3,5
2,bgbl1-2021-74-3,bgbl1,2021,74,2021-10-19T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,3,Erste Verordnung zur Änderung der Bußgeldkatal...,2021-10-13T00:00:00Z,4688.0,8,16
3,bgbl1-2021-73-1,bgbl1,2021,73,2021-10-18T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,1,Gesetz zur Verbesserung der Transparenzregeln ...,2021-10-08T00:00:00Z,4650.0,2,5
4,bgbl1-2021-73-2,bgbl1,2021,73,2021-10-18T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,2,Verordnung zur Stärkung der Organisationen und...,2021-10-11T00:00:00Z,4655.0,7,12


## DIP

Let's download the proposals

The API key until end of May 2022 is: `N64VhW8.yChkBUIJeosGojQ7CSR2xwLf3Qy7Apw464`

Auf Anfrage können Sie auch einen eigenen API Key erhalten, der zeitlich unbefristet ist.

Bitte stellen Sie eine diesbezügliche Anfrage per E-Mail an: infoline.id3@bundestag.de

### technical info about the API

Each call is limited to 50 items in the reply. A reply will also include an item called a 'cursor' which help to keep track of queries that require multiple calls (answers consist of more than 50 items). You keep querying the API with the same parameters, adding the previous cursor you got, until the new cursor matches the one you have, at which point you know there aren't any more items and your query is complete. 

I've implemented this logic with a while loop for now. It's a bit slow but in the future we will only need an update call so I'm not too worried about optimisaiton for now.

In [ ]:
cursor_check = 42
x = 0
dip_response = []
dip_response.append(
    requests.get(
        "https://search.dip.bundestag.de/api/v1/drucksache?f.zuordnung=BT&f.datum.start=2015-01-&apikey=N64VhW8.yChkBUIJeosGojQ7CSR2xwLf3Qy7Apw464"
    )
)
cursor_new = dip_response[-1].json()["cursor"].replace("+", "%2B").replace("/", "%2F")
print(cursor_new)
while cursor_new != cursor_check:
    cursor_check = cursor_new
    dip_response.append(
        requests.get(
            "https://search.dip.bundestag.de/api/v1/drucksache?f.zuordnung=BT&f.datum.start=2015-01&apikey=N64VhW8.yChkBUIJeosGojQ7CSR2xwLf3Qy7Apw464&cursor="
            + cursor_new
        )
    )
    cursor_new = (
        dip_response[-1].json()["cursor"].replace("+", "%2B").replace("/", "%2F")
    )
    x += 1
    print(
        cursor_new, x
    )  # added for testing today because there's an API malefunction. remove when things get back to normal
    if x % 15 == 0:
        time.sleep(1)

In [60]:
# print(x, len(dip_response[-2].json()['documents']))
# >45, 1646

## last call is empty, it returns the same cursor and no documents
## one to last call returns 45 documents
## all calls before that return 100 documents per call
## so..
# 1645 * 100 +45

print("we have 164,545 document titles and linkes")

we have 164,545 document titles and linkes


In [ ]:
dip_response[3].json()["documents"]

In [78]:
dip_response_df_list = [
    pd.DataFrame.from_dict(dip_response[cursor].json()["documents"])
    for cursor in range(0, x + 1)
]

In [77]:
len(dip_response)

1647

In [96]:
dip_response_df_list = [
    pd.json_normalize(dip_response[cursor].json()["documents"])
    for cursor in range(0, x + 1)
]

In [98]:
dip_response_df = pd.concat(dip_response_df_list, ignore_index=True)

In [114]:
dip_response_df.head(10)

,id,drucksachetyp,dokumentart,autoren_anzahl,typ,vorgangsbezug_anzahl,dokumentnummer,wahlperiode,herausgeber,datum,...,fundstelle.dokumentnummer,fundstelle.datum,fundstelle.dokumentart,fundstelle.drucksachetyp,fundstelle.herausgeber,fundstelle.urheber,fundstelle.verteildatum,autoren_anzeige,anlagen,fundstelle.anlagen
0,258151,Antwort,Drucksache,0,Dokument,1,19/32691,19,BT,2021-10-18,...,19/32691,2021-10-18,Drucksache,Antwort,BT,[Bundesregierung],NaN,NaN,NaN,NaN
1,258147,Antwort,Drucksache,0,Dokument,1,19/32689,19,BT,2021-10-15,...,19/32689,2021-10-15,Drucksache,Antwort,BT,[Bundesregierung],NaN,NaN,NaN,NaN
2,258146,Unterrichtung,Drucksache,0,Dokument,1,19/32688,19,BT,2021-10-15,...,19/32688,2021-10-15,Drucksache,Unterrichtung,BT,[Bundesregierung],2021-10-15,NaN,NaN,NaN
3,258145,Antwort,Drucksache,0,Dokument,1,19/32687,19,BT,2021-10-15,...,19/32687,2021-10-15,Drucksache,Antwort,BT,[Bundesregierung],NaN,NaN,NaN,NaN
4,258144,Antwort,Drucksache,0,Dokument,1,19/32684,19,BT,2021-10-15,...,19/32684,2021-10-15,Drucksache,Antwort,BT,[Bundesregierung],NaN,NaN,NaN,NaN
5,258143,Antwort,Drucksache,0,Dokument,1,19/32683,19,BT,2021-10-15,...,19/32683,2021-10-15,Drucksache,Antwort,BT,[Bundesregierung],NaN,NaN,NaN,NaN
6,258142,Antwort,Drucksache,0,Dokument,1,19/32682,19,BT,2021-10-15,...,19/32682,2021-10-15,Drucksache,Antwort,BT,[Bundesregierung],2021-10-20,NaN,NaN,NaN
7,258141,Antwort,Drucksache,0,Dokument,1,19/32681,19,BT,2021-10-15,...,19/32681,2021-10-15,Drucksache,Antwort,BT,[Bundesregierung],2021-10-19,NaN,NaN,NaN
8,258140,Schriftliche Fragen,Drucksache,0,Dokument,0,19/32679,19,BT,2021-10-15,...,19/32679,2021-10-15,Drucksache,Schriftliche Fragen,BT,[],NaN,NaN,NaN,NaN
9,258136,Antwort,Drucksache,0,Dokument,1,19/32678,19,BT,2021-10-14,...,19/32678,2021-10-14,Drucksache,Antwort,BT,[Bundesregierung],2021-10-20,NaN,NaN,NaN


In [115]:
dip_response_df.to_csv(
    index=False, path_or_buf="../data/source_laws/DIP_response_API.csv"
)

None
